In [123]:
from pyspark.sql.types import StructType, StructField, FloatType, LongType, StringType

In [124]:
hdfs_port = "hdfs://orion11:26990"
# data_path = "/nam_s/nam_201501_s*"
data_path = "/nam_s/*"
# data_path = "/sample/nam_tiny*"

In [125]:
feats = []
f = open('../features.txt')
for line_num, line in enumerate(f):
    if line_num == 0:
        # Timestamp
        feats.append(StructField(line.strip(), LongType(), True))
    elif line_num == 1:
        # Geohash
        feats.append(StructField(line.strip(), StringType(), True))
    else:
        # Other features
        feats.append(StructField(line.strip(), FloatType(), True))
    
schema = StructType(feats)

In [126]:
%%time

df = spark.read.format('csv').option('sep', '\t').schema(schema).load(f'{hdfs_port}{data_path}')
# df.take(1)
# print(df.head())

CPU times: user 0 ns, sys: 2.93 ms, total: 2.93 ms
Wall time: 50 ms


In [127]:
hash_prefix = "c6s64"

In [132]:
%%time

dg = df

dg.createOrReplaceTempView("nam_small")

query_str = f'''
SELECT AVG(temperature_surface) AS tmp_avg,
    MIN(temperature_surface) AS tmp_min,
    MAX(temperature_surface) AS tmp_max,
    AVG(categorical_rain_yes1_no0_surface) AS rain_avg,
    FROM_UNIXTIME(Timestamp/1000, 'YYYY-MM') AS year_month
FROM nam_small
WHERE Geohash LIKE "{hash_prefix}%"
GROUP BY year_month
ORDER BY year_month
'''

print(query_str)


SELECT AVG(temperature_surface) AS tmp_avg,
    MIN(temperature_surface) AS tmp_min,
    MAX(temperature_surface) AS tmp_max,
    AVG(categorical_rain_yes1_no0_surface) AS rain_avg,
    FROM_UNIXTIME(Timestamp/1000, 'YYYY-MM') AS year_month
FROM nam_small
WHERE Geohash LIKE "c6s64%"
GROUP BY year_month
ORDER BY year_month

CPU times: user 1.13 ms, sys: 0 ns, total: 1.13 ms
Wall time: 4.5 ms


In [ ]:
%%time

visibilities = spark.sql(query_str).collect()

In [ ]:
visibilities